# Three

Follows from `two`.

Meant to be run in sessions launched from [here](https://github.com/fomightez/rcomp_testenv). 

This is to run all the notebooks so that content produced by code visible.

### STEP #1: Unpack previous results

**REQUIRED FILES**

Besides this notebook (and installed notedown packages) the following must be uploaded to the running binder session:

- `unrun_notebooks_from_RCompanion.tar.gz` from `Two.ipynb` . This contains unrun notebooks I made for each of the pages plus three pertinent lists stored as json.

Unpack that. And bring in the list of urls and files into memory.

In [ ]:
import os
unpacked_example = os.path.join(".","RCompanion_urls_to_get.json")
file_needed = "unrun_notebooks_from_RCompanion.tar.gz"

import sys
if os.path.isfile(unpacked_example):
    sys.stderr.write("\nAppears '{}' has already been unpacked.\n".format(file_needed))
elif os.path.isfile(file_needed):
    !tar xzf {file_needed}
else:
    sys.stderr.write("\n\n*****************ERROR**************************\n"
        "The file '{0}' is needed.\n"
        "Upload '{0}' to this Jupyter session and re-run this cell.\n"
        "*****************ERROR**************************\n".format(file_needed))
    sys.exit(1)
stored_json_files_and_names = [
    "RCompanion_markdowns_made.json",
    "RCompanion_urls_to_get.json",
    'RCompanion_notebooks_made.json'
                            ]
stored_json_files_and_names_to_unpack_as = {k:f'{k.split("_")[0]}_{k.split("_")[1]}' for k in stored_json_files_and_names}
import json
for k,v in stored_json_files_and_names_to_unpack_as.items():
    with open(k, 'r') as f:
        globals()[v] = json.load(f)

## STEP #2: Run each notebook.

In [ ]:
%cd rcomp_nbs

In [ ]:
''' # THIS RUNS NOTEBOOKS WITHOUT HANDLING ANY ERRORS BUT REPORTS
# THE ERRORS HERE.
for nb in RCompanion_notebooks:
    !jupyter nbconvert --to notebook --ExecutePreprocessor.kernel_name=ir --ExecutePreprocessor.timeout=300 --execute {nb} --output={nb}
    sys.stderr.write("The notebook, {}, has been run"
        ".\n".format(nb))
''';

In [ ]:
# Run notebooks with error handling so still run with R despite errors
# and report errors here and in notebook, based on 
# https://nbconvert.readthedocs.io/en/latest/execute_api.html
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbconvert.preprocessors import CellExecutionError

run_path = '.'
ep = ExecutePreprocessor(timeout=300, kernel_name='ir')

unindicated_kernel_str = '''"metadata": {}'''
kernel_str = '''"metadata": {
  "kernelspec": {
   "display_name": "R",
   "language": "R",
   "name": "ir"
  },
  "language_info": {
   "codemirror_mode": "r",
   "file_extension": ".r",
   "mimetype": "text/x-r-source",
   "name": "R",
   "pygments_lexer": "r",
   "version": "3.6.1"
  }
 }'''
needs_kernel_set = []

d_08_msg ='''If you are running notebook d_08, because of the way the \
`rcompanion` library throws an error if it encounters the result `No significant differences.`\
, see https://github.com/cran/rcompanion/blob/eaab0c944c4c1984707a84d9add4b9450a3bf390/R/cldList.r#L98 \
expect the following for the last cell in this notebook:
An error occurred while executing the following cell:
------------------
### Extract lsmeans table
Sum = PT$diffs.lsmeans.table


### Extract comparisons and p-values
  Comparison = rownames(Sum)

P.value   = Sum$'p-value'


### Adjust p-values
P.value.adj = p.adjust(P.value,
                       method =  "none")   

### Fix names of comparisons
Comparison = gsub("-", "- Day", Comparison)


### Produce compact letter display
library(rcompanion)

cldList(comparison = Comparison,
        p.value    = P.value.adj,
        threshold = 0.05)
------------------

Error: No significant differences.
Traceback:

1. cldList(comparison = Comparison, p.value = P.value.adj, threshold = 0.05)
2. stop("No significant differences.", call. = FALSE)
ERROR: Error: No significant differences.
'''

for nb_fn in RCompanion_notebooks:
    with open(nb_fn) as f:
        nb = nbformat.read(f, as_version=4)
    msg = 'Running the notebook "%s".\n' % nb_fn
    sys.stderr.write(msg)
    if nb_fn == "d_08.ipynb":
      sys.stderr.write(d_08_msg)
    try:
        out = ep.preprocess(nb, {'metadata': {'path': run_path}})
    except CellExecutionError as e:
        out = None
        msg = 'Error executing the notebook "%s".\n\n' % nb_fn
        #msg += 'See notebook "%s" for the traceback.' % nb_fn
        print("Error info for {}:\n".format(nb_fn),e)
        #raise
        # Set up for adding the kernel information because those that stop
        # without completion due to error, don't include that 
        needs_kernel_set.append(nb_fn)
    finally:
        with open(nb_fn, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
def rreplace(s, old, new, occurrence):
  '''
  string replace from right side. Like `rsplit` but for `str.replace()`.
  from https://stackoverflow.com/a/2556252/8508004
  '''
  li = s.rsplit(old, occurrence)
  return new.join(li)
# fix so R is kernel for those not completed due to error being encountered
for nb_fn in needs_kernel_set:
    with open(nb_fn, 'r') as input:
        nb_text=input.read()
    nb_text = rreplace(nb_text,unindicated_kernel_str,kernel_str,1)
    with open(nb_fn, mode='w', encoding='utf-8') as f:
        f.write(nb_text)

## Now collect the notebooks.

Run the next cell to archive the run notebooks made from the content.

In [ ]:
archive_file_name = "Rnotebooks_from_RCompanion.tar.gz"
import os
import sys

files_to_archive = RCompanion_notebooks
!tar czf {archive_file_name} {" ".join(files_to_archive)}
%cd ..
!cp rcomp_nbs/{archive_file_name} .
sys.stderr.write("***************************DONE***********************************\n"
    "'{}' generated. Download it.\n"
    "***************************DONE***********************************".format(archive_file_name))

----

----